# Party time #

In this tutorial we will look at the classes that put the "party" in `sipparty`.

"Party" here means a person who wants to make SIP calls (or participate in other types of "Session" via the "Session Initiation Protocol"). `Party` is the highest-level object in `sipparty`.

So, let's see what types of party we can have with `sipparty`.

In [ ]:
from sipparty.parties import AllPartyTypes
AllPartyTypes

In [ ]:
from sipparty.parties import NoMediaSimpleCallsParty
NoMediaSimpleCallsParty

In [ ]:
p1 = NoMediaSimpleCallsParty()
p2 = NoMediaSimpleCallsParty()
p1.listen()